In [1]:
# Vector3 Class
from dataclasses import dataclass
import math

@dataclass(frozen=True)
class Vector3:
    x: float
    y: float
    z: float

    # Vector addition
    def __add__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    # Vector subtraction
    def __sub__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )

    # Scalar multiplication
    def __mul__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x * scalar,
            self.y * scalar,
            self.z * scalar
        )
    
    # Scalar division
    def __truediv__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x / scalar,
            self.y / scalar,
            self.z / scalar
        )

    __rmul__ = __mul__
    __rtruediv__ = __truediv__

    # Dot product
    def dot(self, other: "Vector3") -> float:
        return (
            self.x * other.x +
            self.y * other.y +
            self.z * other.z
        )

    # Cross product
    def cross(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )

    # Magnitude (length)
    def magnitude(self) -> float:
        return math.sqrt(self.dot(self))

    # Unit vector
    def unit_vector(self) -> "Vector3":
        mag = self.magnitude()
        if mag == 0:
            raise ValueError("Cannot normalize a zero vector")
        return self * (1 / mag)

In [2]:
# Semi Major Axis (a)
def compute_sma(mu:float, energy:float):
    return -mu/(2*energy)


In [3]:
# Eccentricity (e)
import math

def compute_ecc(energy: float, mu: float, h:float):
    return math.sqrt(1+(2*math.pow(h, 2)*energy)/(math.pow(mu, 2)))

In [4]:
# Inclination (i)
import math

def compute_inc(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return math.acos(Z_.dot(h_))


In [5]:
# RAAN (capital omega) Ω
import math

def compute_raan(Nx: float, Ny: float):
    return math.atan2(Ny, Nx)

In [6]:
# Argument of Perigeee (omega sub p) ωp
import math

def compute_argp(h_: Vector3, N_: Vector3, B_: Vector3):
    h_
    N_
    B_
    return math.atan2(h_.dot(N_.cross(B_)), N_.dot(B_))



In [7]:
# True Anomaly (nu) ν
import math

def compute_ta(v_: Vector3, r_: Vector3, B_: Vector3):
    cosTa = r_.dot(B_)/(r_.magnitude()*B_.magnitude())
    ta = math.acos(cosTa)
    if r_.dot(v_) < 0:
        ta = 2*math.pi - ta
    return ta

In [8]:
# Period (TP)
import math

def compute_period(a: float, mu: float):
    return 2*math.pi*math.sqrt(math.pow(a, 3)/mu)

In [9]:
# Apogee (r sub a) ra
def compute_apogee(a: float, e: float):
    return a*(1+e)

In [10]:
# Perigee (r sub p) rp
def compute_perigee(a: float, e: float):
    return a*(1-e)

In [11]:
# Energy (xi) ξ
import math

def compute_energy(v: float, r: float, mu: float):
    return math.pow(v, 2)/2 - mu/r

In [12]:
# Angular Momentum Vector (h_)
def compute_angular_momentum_vector(r: Vector3, v: Vector3):
    return r.cross(v)

In [13]:
# RAAN Vector (N_) Unitized
def compute_raan_vector(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return (Z_.cross(h_)/(Z_.cross(h_).magnitude())).unit_vector()

In [14]:
# Perigee Vector (B_) Unitized
def compute_perigee_vector(mu: float, r_: Vector3, v_: Vector3, h_: Vector3):
    return v_.cross(h_)-mu*(r_/r_.magnitude()).unit_vector()

In [15]:
# Keplerian Elements Class
import math

class KeplerianElements:
    """
    Computes classical Keplerian orbital elements from
    position and velocity state vectors.
    """

    mu_earth = 3.986004418e14  # m^3 / s^2

    def __init__(self, position: Vector3, velocity: Vector3):
        """
        Parameters
        ----------
        position : Vector3
            Position vector (meters)
        velocity : Vector3
            Velocity vector (m/s)
        """

        mu = self.mu_earth
        k_hat = Vector3(0, 0, 1)

        # Specific orbital energy
        self.xi = compute_energy(velocity.magnitude(), position.magnitude(), mu)

        # Semi-major axis
        self.a = compute_sma(mu, self.xi)

        # Angular momentum vector
        self.h_ = compute_angular_momentum_vector(position, velocity)

        # Eccentricity
        self.ecc = compute_ecc(self.xi, mu, self.h_.magnitude())

        # Inclination
        self.inc = compute_inc(k_hat, self.h_)
        self.inc_deg = math.degrees(self.inc)

        # RAAN
        self.N_Unit = compute_raan_vector(k_hat, self.h_)
        self.raan = compute_raan(self.N_Unit.x, self.N_Unit.y)
        self.raan_deg = math.degrees(self.raan)

        # Argument of perigee
        self.B_Unit = compute_perigee_vector(
            mu,
            position,
            velocity,
            self.h_
        )
        self.argp = compute_argp(self.h_, self.N_Unit, self.B_Unit)
        self.argp_deg = math.degrees(self.argp)

        # True anomaly
        self.ta = compute_ta(velocity, position, self.B_Unit)
        self.ta_deg = math.degrees(self.ta)

        # Orbital period (elliptical only)
        self.period = compute_period(self.a, mu)

        # Apoapsis / periapsis
        self.apogee = compute_apogee(self.a, self.ecc)
        self.perigee = compute_perigee(self.a, self.ecc)


```^ Previous Lessons ^```

# Problem 1:

In [16]:
pos = Vector3(326151.080726, 6077471.251787, 2944583.918767)  # meters
vel = Vector3(-7455.178720, -482.482572, 1910.883434) # m/s

kepler_elements = KeplerianElements(pos, vel)
print("nu (true anomaly) is: " + str(kepler_elements.ta) + " rad")

nu (true anomaly) is: 0.533708002792791 rad


In [17]:
# Eccentric Anomaly (E)
import math

def compute_eccentric_anomaly(nu: float, e: float) -> float:
    return math.asin((math.sin(nu)*math.sqrt(1 - math.pow(e, 2))) / (1 + e*math.cos(nu)))


In [18]:
E_SV = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
print("E_SV (eccentric anomaly) is: " + str(E_SV) + " rad")

E_SV (eccentric anomaly) is: 0.5286424011417852 rad


In [19]:
# Mean Anomaly (M)
import math

def compute_mean_anomaly(E: float, e: float) -> float:
    return E - e*math.sin(E)

In [20]:
M_SV = compute_mean_anomaly(E_SV, kepler_elements.ecc)
print("M0 (mean anomaly) is: " + str(M_SV) + " rad")

M0 (mean anomaly) is: 0.5235987858960514 rad


In [21]:
# Mean Motion (n)
import math

def compute_mean_motion(mu: float, a: float):
    # note a is semi major axis in meters
    return math.sqrt(mu/math.pow(a,3))

##### Time of flight from perigee to nu (true anomaly)
Initially:  
nu (true anomaly) is: 0.533708002792791 rad  
E (eccentric anomaly) is: 0.5286424011417852 rad  
M (mean anomaly) is: 0.5235987858960514 rad  
  
At Perigee:  
tp = 0  
Ep = 0  

General Time of Flight Equation:  
`t-t0 = kTP + (1/n)(E - e * sin(E)) - (1/n) * (E0 - e * sin(E0))`  
- T = Time of last perigee passage  
- t0 = Time from perigee to E0  
- t = Time from perigee to E  
- k = perigee passages between E0 and E  
- k * TP = number of full orbits from epoch  

Therefore  
t0 is 0 and k is 0 and E0 is 0 so  
`t = (1/n)(E - e*sin(E))`

In [22]:
n = compute_mean_motion(kepler_elements.mu_earth, kepler_elements.a)
print("n (mean motion) is: " + str(n) + " rad/sec") 

n (mean motion) is: 0.0011209657051213528 rad/sec


In [23]:
t_delta_perigeee_to_nu = (1/n)*(E_SV-kepler_elements.ecc * math.sin(E_SV))
print("time delta between perigee and nu: " + str(t_delta_perigeee_to_nu) + " sec")

time delta between perigee and nu: 467.0961685124594 sec


##### Time of flight from nu to nu = 65deg
Initially:  
nu (true anomaly) is: 0.533708002792791 rad  
E (eccentric anomaly) is: 0.5286424011417852 rad  
M (mean anomaly) is: 0.5235987858960514 rad  
  
At nu = 65deg:  
calculate E

General Time of Flight Equation:  
`t-t0 = kTP + (1/n)(E - e * sin(E)) - (1/n) * (E0 - e * sin(E0))`  
- T = Time of last perigee passage  
- t0 = Time from perigee to E0  
- t = Time from perigee to E  
- k = perigee passages between E0 and E  
- k * TP = number of full orbits from epoch  

Therefore  
t0 is 0 and k is 0 and E0 is 0.5286424011417852 so  
`t = (1/n)(E_65-e*sin(E65)) - (1/n)*(E0-e*sin(E0))`

In [24]:
E_65 = compute_eccentric_anomaly(math.radians(65), kepler_elements.ecc)
t_delta_nu_to_65 = (1/n)*(E_65-kepler_elements.ecc*math.sin(E_65)) - (1/n)*(E_SV-kepler_elements.ecc*math.sin(E_SV))
print("E_65 is " + str(E_65))
print("time delta between nu and 65 deg: " + str(t_delta_nu_to_65) + " sec")

E_65 is 1.1254198827627566
time delta between nu and 65 deg: 528.8267149213564 sec


##### Verification
Given 528.8267149213564 sec, now I need to verify that after that time, we will be at nu=65 deg

Time to solve for E by Newton-Raphson
`E0 = M` (mean anomaly. Mean Motion is constant (rad/sec), Mean anomaly is not)
M for Newton-Raphson is: `M = n(t-T)` (essentially rad/sec*sec which gives you rad)
remember the formula for `E_k_1=E_k + (M-(E_k-e*sin(E_k)))/(1-e*cos(E_k))`  
Solve for nu after you get E using `cos(nu) = (e-cos(E))/(e*cos(E)-1)`

In [41]:
import math

# compute E based on time delta
M_propagated = M_SV + n*t_delta_nu_to_65
E_k = M_propagated
E_k_1 = 100000000000 # just to make sure our diff is large enough for first iteration
count = 0
while (abs(E_k-E_k_1) != 0):
    E_k = E_k_1
    E_k_1=E_k + (M_propagated-(E_k-kepler_elements.ecc*math.sin(E_k)))/(1-kepler_elements.ecc*math.cos(E_k))
    count = count + 1
    print("Iteration " + str(count) + ": " + str(abs(E_k-E_k_1)))
E_check = E_k_1%(2*math.pi)
print("Eccentric Anomaly Propagated: " + str(E_check) + " rad") # mod by 2pi in case k is greater than 0
print("k: " + str(math.floor((E_k_1-E_SV))/(2*math.pi)))
print("Diff between E_65 and computed E_65 through propagation is: " + str(abs(E_65-E_check)))


Iteration 1: 100372228187.19342
Iteration 2: 375802045.9425468
Iteration 3: 3565896.24297107
Iteration 4: 7950.319210706047
Iteration 5: 11.178083766057082
Iteration 6: 0.11661818978111382
Iteration 7: 5.913950939429036e-05
Iteration 8: 1.5849987988758585e-11
Iteration 9: 0.0
Eccentric Anomaly Propagated: 1.1254198827627564 rad
k: 0.0
Diff between E_65 and computed E_65 through propagation is: 2.220446049250313e-16


In [42]:
import math

# compute true anomaly from E
nu_check_cos = (kepler_elements.ecc-math.cos(E_check))/(kepler_elements.ecc*math.cos(E_check)-1)
nu_check_sin = (math.sin(E_check)*math.sqrt(1-math.pow(kepler_elements.ecc, 2)))/(1-kepler_elements.ecc*math.cos(E_check))

# ensure we land in the correct quadrant:
nu_check = math.atan2(nu_check_sin, nu_check_cos)
if nu_check < 0:
    nu_check = nu_check + 2*math.pi
print("nu propagated: " + str(math.degrees(nu_check)) + " deg")

nu propagated: 64.99999999999997 deg


##### Starting at Vo, what is the true anomaly after 2700 seconds?

In [45]:
# Propagate nu given delta t
import math

def propagate_nu_given_delta_t(kepler_elements: KeplerianElements, time_delta: float) -> tuple[float, int, float]:
    E0 = compute_eccentric_anomaly(kepler_elements.ta, kepler_elements.ecc)
    M0 = compute_mean_anomaly(E0, kepler_elements.ecc)
    M_propagated = M0 + n*time_delta
    E_k = M_propagated
    E_k_1 = 100000000000 # just to make sure our diff is large enough for first iteration
    while (abs(E_k-E_k_1) != 0):
        E_k = E_k_1
        E_k_1=E_k + (M_propagated-(E_k-kepler_elements.ecc*math.sin(E_k)))/(1-kepler_elements.ecc*math.cos(E_k))
    E_propagated = E_k_1%(2*math.pi)
    k = math.floor((E_k_1-E0)/(2*math.pi))
    cos_nu = (kepler_elements.ecc-math.cos(E_propagated))/(kepler_elements.ecc*math.cos(E_propagated)-1)
    sin_nu = (math.sin(E_propagated)*math.sqrt(1-math.pow(kepler_elements.ecc, 2)))/(1-kepler_elements.ecc*math.cos(E_propagated))
    nu_propagated = math.atan2(sin_nu, cos_nu)
    # ensure we land in the correct quadrant:
    nu_propagated = math.atan2(math.sin(nu_propagated), math.cos(nu_propagated))
    if nu_propagated < 0:
        nu_propagated = nu_propagated + 2*math.pi
    return E_propagated, k, nu_propagated

In [47]:
# calculate nu after 2700 seconds:
E_propagated, k, nu_propagated_2700 = propagate_nu_given_delta_t(kepler_elements, 2700)
print("Propagated E (2700): " + str(E_propagated))
print("Propagated nu (2700): " + str(nu_propagated_2700))
print("k (2700): " + str(k))


Propagated E (2700): 3.546268977283855
Propagated nu (2700): 3.5423496855494134
k (2700): 0
